In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
!curl -O https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4764M  100 4764M    0     0  8625k      0  0:09:25  0:09:25 --:--:-- 8908k


In [3]:
#Read the dataset
import tarfile
dataset = tarfile.open('food-101.tar.gz')

In [4]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','image_file'])

In [5]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, name]

In [6]:
#extract data
dataset.extractall(path='')

In [8]:
!ls -l food-101/images

total 3616
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 apple_pie
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 baby_back_ribs
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 baklava
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 beef_carpaccio
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 beef_tartare
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 beet_salad
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 beignets
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 bibimbap
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 bread_pudding
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 breakfast_burrito
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 bruschetta
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 caesar_salad
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 cannoli
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 caprese_salad
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 carrot_cake
drwxr-xr-x 2 3156 320 36864 Jul  9  2014 ceviche
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 cheesecake
drwxr-xr-x 2 3156 320 36864 Sep 21  2013 cheese_plate
drwxr-xr-x 2 3156 320 36864 Sep 21  201

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
train_df.to_csv('food-101/images/train.csv',index=False)
test_df.to_csv('food-101/images/test.csv', index=False)

In [12]:
#Read training and test Dataframe
train_df = pd.read_csv('food-101/images/train.csv')
test_df = pd.read_csv('food-101/images/test.csv')

In [13]:
#Check contents
train_df.sample(n=5)

,class,image_file
58540,pad_thai,food-101/images/pad_thai/1375494.jpg
67303,croque_madame,food-101/images/croque_madame/2174524.jpg
27517,apple_pie,food-101/images/apple_pie/2517059.jpg
27282,guacamole,food-101/images/guacamole/3453560.jpg
60870,donuts,food-101/images/donuts/3420049.jpg


In [15]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Food classes: ', class_names)
len(class_names)

Food classes:  ['dumplings', 'bruschetta', 'churros', 'croque_madame', 'spaghetti_carbonara', 'fried_calamari', 'bread_pudding', 'panna_cotta', 'frozen_yogurt', 'club_sandwich', 'filet_mignon', 'pho', 'chicken_curry', 'sushi', 'ravioli', 'donuts', 'beef_tartare', 'omelette', 'french_onion_soup', 'huevos_rancheros', 'french_fries', 'chocolate_mousse', 'pad_thai', 'beet_salad', 'caesar_salad', 'fish_and_chips', 'cup_cakes', 'breakfast_burrito', 'chicken_quesadilla', 'ramen', 'pulled_pork_sandwich', 'risotto', 'foie_gras', 'hummus', 'ceviche', 'crab_cakes', 'grilled_salmon', 'macaroni_and_cheese', 'carrot_cake', 'seaweed_salad', 'pork_chop', 'nachos', 'peking_duck', 'macarons', 'spaghetti_bolognese', 'tacos', 'samosa', 'caprese_salad', 'chicken_wings', 'paella', 'chocolate_cake', 'baby_back_ribs', 'cheesecake', 'miso_soup', 'onion_rings', 'red_velvet_cake', 'grilled_cheese_sandwich', 'escargots', 'pancakes', 'spring_rolls', 'apple_pie', 'lobster_roll_sandwich', 'baklava', 'french_toast', 

101

In [17]:
#Define some parameters
img_size = 224
img_depth = 3  

In [19]:
def normalize_data(img):
    
    #Normalize for ResNet50
    return tf.keras.applications.resnet50.preprocess_input(img)

In [20]:
#Define Training Data Generator with augmentations
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                preprocessing_function=normalize_data) #Normalize the data accordingly

#Define Test Data Generator with NO augmentations
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=normalize_data) #Normalize the data accordingly

In [22]:
#Training (from dataframe)
train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='image_file', #File path for image
                                                    y_col='class',           #Class for the image
                                                    target_size=(img_size, img_size), #Image resize dimensions
                                                    batch_size=800)

Found 80800 validated image filenames belonging to 101 classes.


In [23]:
#Test (from dataframe)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col='image_file', #File path for image
                                                  y_col='class',           #Class for the image
                                                  target_size=(img_size, img_size), #Image resize dimensions
                                                  batch_size=800)

Found 20200 validated image filenames belonging to 101 classes.


In [24]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(224,224,3),
                                       weights='imagenet')

94773248/94765736 [==============================] - 3s 0us/step


In [28]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [30]:
model.output

<tf.Tensor 'conv5_block3_out/Relu:0' shape=(None, 7, 7, 2048) dtype=float32>

In [31]:
#get Output layer of Pre0trained model
x = model.output

#Global average pool to reduce number of features and Flatten the output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [32]:
#Output shape of Global Average Pooling
x

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 2048) dtype=float32>

In [33]:
#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)

In [34]:
prediction

<tf.Tensor 'dense/Softmax:0' shape=(None, 101) dtype=float32>

In [35]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [36]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
#How does our overall model looks
final_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [38]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('food_resnet.h5', # adreess or location will also work here
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

In [42]:
final_model.fit_generator(train_generator, 
                          epochs=1,
                          steps_per_epoch= 80800//5050, # len(training_size/batch size)
                          validation_data=test_generator,
                          validation_steps = 20200//5050, 
                          callbacks=[model_checkpoint])

16/16 [==============================] - ETA: 0s - loss: 1.8624 - accuracy: 0.5364 WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 252 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_accuracy improved from -inf to 0.54010, saving model to food_resnet.h5
16/16 [==============================] - 333s 21s/step - loss: 1.8624 - accuracy: 0.5364 - val_loss: 1.8185 - val_accuracy: 0.5401


In [ ]:
#Lets train for 3 more steps
final_model.fit_generator(train_generator, 
                          epochs=5,
                          initial_epoch=1,
                          steps_per_epoch= 80800//5050,
                          validation_data=test_generator,
                          validation_steps = 20200//5050, 
                          callbacks=[model_checkpoint])

Epoch 2/5
16/16 [==============================] - ETA: 0s - loss: 1.7968 - accuracy: 0.5508 
Epoch 00002: val_accuracy improved from 0.54010 to 0.54625, saving model to food_resnet.h5
16/16 [==============================] - 242s 15s/step - loss: 1.7968 - accuracy: 0.5508 - val_loss: 1.7902 - val_accuracy: 0.5462
Epoch 3/5
16/16 [==============================] - ETA: 0s - loss: 1.7678 - accuracy: 0.5601 
Epoch 00003: val_accuracy improved from 0.54625 to 0.55250, saving model to food_resnet.h5
16/16 [==============================] - 235s 15s/step - loss: 1.7678 - accuracy: 0.5601 - val_loss: 1.7680 - val_accuracy: 0.5525
Epoch 4/5
16/16 [==============================] - ETA: 0s - loss: 1.7279 - accuracy: 0.5688 

In [ ]:
final_model.save('mymodel.h5')

In [ ]:
model = tf.keras.models.load_model('mymodel.h5')